# DenseNet121 Image Classifier

**This notebook should run correctly both in colab and on jupyter.**
- Uses stratified sampling (making sure all classes have same ratio in all sets).
- Uses metadata from **'data/encoded_columns_data'**.
- Uses images from folder **'directly_processed_images'**.

## **Setup**

In [ ]:
import sys
on_colab = 'google.colab' in sys.modules
if on_colab:
  print("Running notebook on google colab")
  # use ice cream to conveniently print stuff
  !pip install icecream
else:
  print("Notebook not running on google colab")

Running notebook on google colab


In [ ]:
# imports
from icecream import ic
import pandas as pd
import numpy as np
import torch
import time
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
import timeit
import torchvision
# import torchsummary
from torchvision import transforms, datasets
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
# from torchsummary import summary
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from PIL import Image

import matplotlib.pyplot as plt
import seaborn as sns

# to save session
import pickle

from tqdm import tqdm

In [ ]:
if on_colab:
  from google.colab import drive
  drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
print("PyTorch version:")
print(torch.__version__)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using {} device".format(device))

PyTorch version:
2.2.1+cu121
Using cuda device


## Define paths to image folder & metadata

In [ ]:
# paths
if on_colab:
  metadata_path = '/content/drive/MyDrive/CDS Project/data/encoded_final_data.csv'
  images_path = '/content/drive/MyDrive/CDS Project/directly_processed_images'
else:
  metadata_path = '../data/encoded_final_data.csv'
  images_path = '../directly_processed_images'

In [ ]:
metadata_df = pd.read_csv(metadata_path)
metadata_df

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/CDS Project/data/encoded_final_data.csv'

In [ ]:
num_images = len([file for file in os.listdir(images_path) if os.path.isfile(os.path.join(images_path, file))])
print("Number of images: ", num_images)

## Train test split (Stratified Sampling)

In [ ]:
# Split data in features and target:
X = metadata_df  # features
y = metadata_df['target']  # target

# Splitting into train, test, and validation sets with stratified sampling
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=0, stratify=y_temp)

ic(len(X_train), len(X_test), len(X_val))

In [ ]:
# Resizing to 224x224, some transformations/augmentation
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.RandomCrop(224, padding=4),
    # transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2), # random adjustments
    # transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),  # for blurry images
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # standard normalization
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

## Define CustomDataset function

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, metadata, transform=None):
        self.metadata = metadata
        self.images_path = images_path
        self.transform = transform

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        img_id = self.metadata.iloc[idx, 0]  # Get image ID of sample using the metadata index
        img_name = os.path.join(self.images_path, f'processed_{img_id}.jpg')  # name of image files
        if not os.path.exists(img_name):
            raise FileNotFoundError(f"Image file not found: {img_name}")

        image = Image.open(img_name)
        row = metadata_df[metadata_df['image_id'] == img_id]
        label = row.iloc[0]['target']

        if self.transform is not None:
            image = self.transform(image)

        # # extract demographic features
        # age = torch.tensor(self.metadata.iloc[idx, 1])
        # localization = torch.tensor(self.metadata.iloc[idx, 3])
        # gender = torch.tensor(self.metadata.iloc[idx, 4])

        return image, label

## Define train, validation and test datasets/dataloaders

In [ ]:
# custom datasets
train_dataset = CustomDataset(metadata=X_train, transform=train_transform)
val_dataset = CustomDataset(metadata=X_val, transform=val_transform)
test_dataset = CustomDataset(metadata=X_test, transform=val_transform)

# Create data loaders (optimized for faster loading)
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8, persistent_workers=True, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=8, persistent_workers=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=8, persistent_workers=True, pin_memory=True)

# check
ic(len(train_dataset), len(val_dataset), len(test_dataset))

In [ ]:
# test to check some samples in split sets
for index in range(20):

  image, label = train_dataset[index]

  # Display information about the sample
  # print("Random Sample Index:", index)
  # print("Image Shape:", image.shape)
  # print("Label:", label)
  # print(type(label))

  # Display the image
  plt.imshow(image.permute(1, 2, 0))  # permute to rearrange dimensions from CxHxW to HxWxC
  plt.title(f"Label: {label}")
  plt.axis('off')
  plt.show()

## DenseNet121 + modified last layer

In [ ]:
model_densenet = models.densenet121(pretrained=True)

#### Modify last layer
- adjust to the appropriate number of output classes (7)

In [ ]:
num_features = model_densenet.classifier.in_features
model_densenet.classifier = nn.Linear(num_features, 7)

model_densenet # new model structure

#### Adam optimizer + CEL

In [ ]:
# Loss function: CEL
criterion = nn.CrossEntropyLoss()

# Optimizer: Adam
optimizer = torch.optim.Adam(model_densenet.parameters(), lr=0.001, )

### Training

In [ ]:
model_densenet.to(device)
criterion.to(device)

In [ ]:
# Training for one epoch

def train_model(model, criterion, optimizer, train_loader, val_loader):

    model.train()
    train_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in tqdm(train_loader):
        labels = labels.long()
        labels = labels.to(device)
        images = images.to(device)
        optimizer.zero_grad()
        # forward propagation, i.e. get predictions
        outputs = model(images)
        # calculate loss and backpropagate to model paramters:
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    train_loss /= len(train_loader.dataset)
    train_acc = correct_train / total_train

    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in val_loader:
            labels=labels.long()
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_loss /= len(val_loader.dataset)
    val_acc = correct_val / total_val

    return train_loss, val_loss, train_acc, val_acc

In [ ]:
# specify epochs to train
total_epochs = 40

tl = []
vl = []
ta = []
va = []

for epoch in range(total_epochs):
    start_time = time.time()  # Record start time

    train_loss, val_loss, train_acc, val_acc = train_model(model_densenet, criterion, optimizer, train_loader, val_loader)
    tl.append(train_loss)
    vl.append(val_loss)
    ta.append(train_acc)
    va.append(val_acc)

    elapsed_time = time.time() - start_time  # Calculate elapsed time

    print(f"Epoch {epoch+1}/{total_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Elapsed Time: {elapsed_time:.2f} seconds")

In [ ]:
# extra epochs (if needed)

for epoch in range(10):
    start_time = time.time()  # Record start time

    train_loss, val_loss, train_acc, val_acc = train_model(model_densenet, criterion, optimizer, train_loader, val_loader)
    tl.append(train_loss)
    vl.append(val_loss)
    ta.append(train_acc)
    va.append(val_acc)

    elapsed_time = time.time() - start_time  # Calculate elapsed time

    print(f"Epoch {40+epoch+1}/{50}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Elapsed Time: {elapsed_time:.2f} seconds")

### Visualisation

In [ ]:
# visualisation function
def plotResults():
    plt.figure(figsize=(10,6))
    epovec=range(len(ta))
    plt.plot(epovec,ta,epovec,tl,epovec,va,epovec,vl,linewidth=3)
    plt.legend(('Training Accuracy','Training Loss','Validation Accuracy','Validation Loss'))

    # make the graph understandable:
    plt.title("Losses and Accuracies")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.show()

In [ ]:
# plot loss function
def plotLosses():
    plt.figure(figsize=(10,6))
    epovec=range(len(tl))
    plt.plot(epovec,tl,epovec,vl,linewidth=3)
    plt.legend(('Training Loss', 'Validation Loss'))

    # make the graph understandable:
    plt.title("Training and Validation Losses")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss")
    plt.show()

In [ ]:
# plot accuracies function
def plotAccuracies():
    plt.figure(figsize=(10,6))
    epovec=range(len(ta))
    plt.plot(epovec,ta,epovec,va,linewidth=3)
    plt.legend(('Training Accuracy', 'Validation Accuracy'))

    # make the graph understandable:
    plt.title("Training and Validation Accuracies")
    plt.xlabel("Epoch #")
    plt.ylabel("Accuracy")
    plt.show()

In [ ]:
# evaluate model function
def evaluate_model(model, criterion, data_loader):
    model.eval()
    loss = 0.0
    correct = 0
    total = 0

    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for images, labels in data_loader:
            labels = labels.long()
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss += criterion(outputs, labels).item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    loss /= len(data_loader.dataset)
    accuracy = correct / total

    return loss, accuracy, all_predictions, all_labels

In [ ]:
plotResults()

In [ ]:
# plot losses separately
plotLosses()

In [ ]:
# plot accuracies separately
plotAccuracies()

### Evaluation on Test Set

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
import matplotlib.pyplot as plt
import numpy as np

# Evaluate model
test_loss, test_accuracy, all_predictions, all_labels = evaluate_model(model_densenet, criterion, test_loader)

# Compute classification report
report = classification_report(all_labels, all_predictions)
print("Classification Report:\n", report)

# Compute confusion matrix
conf_matrix = confusion_matrix(all_labels, all_predictions)
print("Confusion Matrix:\n", conf_matrix)

# Compute accuracy per class
class_accuracy = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
print("Accuracy per Class:", class_accuracy)
print("Test accuracy: ", test_accuracy)

### Save model & parameters

In [ ]:
# define saving paths
if on_colab:
  save_to_path = '/content/drive/MyDrive/CDS Project/saved_models/densenet_classifier_40epochs.pth'
  save_params_path = '/content/drive/MyDrive/CDS Project/saved_models/densenet_classifier_40epochs_params.pth'
else:
  save_to_path = '../saved_models/densenet_classifier_40epochs.pth'
  save_params_path = '../saved_models/densenet_classifier_40epochs_params.pth'

In [ ]:
torch.save(model_densenet, save_to_path)
torch.save(model_densenet.state_dict(), save_params_path)